# Day 5

Let's create a product that builds a brochure for a company to be used for prospective clients, investors and potential recruiters.

In [ ]:
# Setting up Gemini and importing necessary libraries
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from scraper import fetch_website_links, fetch_website_contents


load_dotenv(override=True)

api_key = os.getenv("GEMINI_API_KEY")
base_url = os.getenv("GEMINI_BASE_URL", "https://generativelanguage.googleapis.com/v1beta/openai/")
model = os.getenv("GEMINI_MODEL", "gemini-3-flash-preview")

if not api_key:
    raise ValueError("No GEMINI_API_KEY found in .env file")
else:
    print("API key found!")
    client = OpenAI(base_url=base_url, api_key=api_key)

In [ ]:
# Get the links from a webpage
links = fetch_website_links("https://manuel-valles.com")
links

## 1. Make AI to figure out which links are relevant
Our AI model should decide which links are relevant, and replace relative links such as `/about` with `https://company.com/about`. We will use _one shot prompting_ in which we provide an example of how it should respond in the prompt (e.g. JSON):

In [ ]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
print(get_links_user_prompt("https://manuel-valles.com"))

In [ ]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {model} model...")
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links:")
    return links

In [ ]:
select_relevant_links("https://manuel-valles.com")

## 2. Make the brochure
We can add more details into our AI like the general content and the content for each link/page.

In [ ]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
print(fetch_page_and_all_relevant_links("https://manuel-valles.com"))

In [ ]:
# System prompt for brochure creation 
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [ ]:
# User prompt for brochure creation
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5k characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("Wave Architects", "https://wavearchitects.co.uk")

In [ ]:
def create_brochure(company_name, url):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("Wave Architects", "https://wavearchitects.co.uk")

In [ ]:
# Streaming brochure creation. This streams the response and updates the display in real-time, like a typing effect
def stream_brochure(company_name, url):
    stream = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("Wave Architects", "https://wavearchitects.co.uk")